In [1]:
DATASET_PATH = "/content/dataset"
DATASET_TYPE = "ravdess"


In [2]:
# =============================================================
# ✅ FIX ENVIRONMENT: Use Python 3.10 (Compatible with TF & NumPy)
# =============================================================

!sudo apt-get update -y > /dev/null
!sudo apt-get install python3.10 python3.10-dev python3.10-distutils -y > /dev/null

# 🔄 Switch Python version
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1
!sudo apt-get install python3-pip -y > /dev/null

# 🧠 Upgrade pip and reinstall key packages
!python3 -m pip install -U pip setuptools wheel --quiet
!python3 -m pip install tensorflow==2.15.0 keras==2.15 numpy==1.26.4 librosa==0.10.1 scikit-learn==1.5.2 soundfile matplotlib tqdm pydub ffmpeg-python --quiet

# ✅ Verify
import tensorflow as tf, numpy, librosa, sklearn
print("✅ TensorFlow:", tf.__version__)
print("✅ NumPy:", numpy.__version__)
print("✅ Librosa:", librosa.__version__)


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 4.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 62.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.1

In [3]:
# ============================================================
# STEP 1: Import libraries and setup environment
# ============================================================
import tensorflow as tf
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from google.colab import files
from pydub import AudioSegment
import os
import soundfile as sf

print("✅ TensorFlow:", tf.__version__)
print("✅ NumPy:", np.__version__)
print("✅ Librosa:", librosa.__version__)


✅ TensorFlow: 2.19.0
✅ NumPy: 2.0.2
✅ Librosa: 0.11.0


/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


In [4]:
# ============================================================
# STEP 2: Upload your audio file (any format: mp4, wav, mp3, opus, ogg)
# ============================================================
from google.colab import files
from pydub import AudioSegment
import os
import subprocess

uploaded = files.upload()

file_name = list(uploaded.keys())[0]
print(f"📁 Uploaded file: {file_name}")

ext = file_name.split('.')[-1].lower()
print(f"🎧 Detected format: {ext}")

wav_path = "converted_audio.wav"

try:
    if ext in ['opus', 'ogg']:
        print("🔄 Using ffmpeg to convert .opus/.ogg to .wav ...")
        subprocess.run(["ffmpeg", "-i", file_name, wav_path, "-y"], check=True)
    else:
        audio = AudioSegment.from_file(file_name, format=ext)
        audio.export(wav_path, format="wav")
    print("✅ Successfully converted to WAV:", wav_path)
except Exception as e:
    print("❌ Conversion failed:", e)


Saving angry.opus to angry.opus
📁 Uploaded file: angry.opus
🎧 Detected format: opus
🔄 Using ffmpeg to convert .opus/.ogg to .wav ...
✅ Successfully converted to WAV: converted_audio.wav


In [5]:
# ============================================================
# STEP 3: Extract audio features (MFCCs)
# ============================================================
def extract_features(file_path):
    data, sample_rate = librosa.load(file_path, duration=3, offset=0.5)
    mfccs = librosa.feature.mfcc(y=data, sr=sample_rate, n_mfcc=40)
    mfccs_scaled = np.mean(mfccs.T, axis=0)
    return mfccs_scaled

features = extract_features(wav_path)
print("✅ Extracted feature shape:", features.shape)


✅ Extracted feature shape: (40,)


In [6]:
# ============================================================
# STEP 4: Define model architecture (simple CNN)
# ============================================================
model = Sequential([
    Dense(256, activation='relu', input_shape=(40,)),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(3, activation='softmax')  # 3 classes: Happy, Sad, Angry
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 🧠 Simulated random weights (for demo)
# In real usage, load trained model weights via model.load_weights('model.h5')
print("✅ Model ready.")


✅ Model ready.


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
# ============================================================
# STEP 5: Make prediction
# ============================================================
# Expand dimensions to match model input
feature_input = np.expand_dims(features, axis=0)

# Simulate a random output (replace with model.predict(feature_input) when trained)
predicted_probs = np.random.rand(1, 3)
predicted_probs /= np.sum(predicted_probs)  # normalize to sum=1

# Labels
emotions = ['Happy', 'Sad', 'Angry']

# Get the top prediction
predicted_index = np.argmax(predicted_probs)
predicted_emotion = emotions[predicted_index]

print(f"🎯 Predicted Emotion: {predicted_emotion}")
print(f"📊 Confidence Scores: {dict(zip(emotions, predicted_probs[0].round(3)))}")


🎯 Predicted Emotion: Sad
📊 Confidence Scores: {'Happy': np.float64(0.43), 'Sad': np.float64(0.522), 'Angry': np.float64(0.048)}
